# Стохастический градиентный и координатный спуски

Для каждого задания указано количество баллов (если они оцениваются отдельно) + 1 балл за аккуратное и полное выполнение всего задания

## Загрузка и подготовка данных

**Загрузите уже знакомый вам файл *Advertising.csv* как объект DataFrame.** 

In [ ]:
#ваш код 
import numpy as np
import pandas as pd
df = pd.read_csv('Advertising.csv', delimiter=',')
df

**Проверьте, есть ли в данных пропуски и, если они есть - удалите их**

In [4]:
#ваш код
df.isnull().any().any()

False

**Преобразуйте ваши признаки в массивы NumPy и разделите их на переменные X (предикторы) и y(целевая переменная)** 

In [23]:
#ваш код
x = df[['TV', 'radio', 'newspaper']].to_numpy()

y = df['sales'].to_numpy()

## Координатный спуск (3 балла)

**Добавим единичный столбец для того, чтобы у нас был свободный коэффициент в уравнении регрессии:**

In [27]:
import numpy as np
X = np.hstack([np.ones(x.shape[0]).reshape(-1, 1), x])

**Нормализуем данные: обычно это необходимо для корректной работы алгоритма**

In [28]:
X = X / np.sqrt(np.sum(np.square(X), axis=0))


**Реализуйте алгоритм координатного спуска:** (3 балла)

Ниже приведен алгоритм:

<a href="https://ibb.co/Th3BQFn"><img src="https://i.ibb.co/DK2DBS6/zascas.jpg" alt="zascas" border="0"></a>

Примечание: 1000 итераций здесь указаны для этого задания, на самом деле их может быть намного больше, нет детерменированного значения.

Вам необходимо реализовать координатный спуск, и вывести веса в модели линейной регрессии.

In [ ]:
#ваш код
w = np.zeros(X.shape[1])
 
for iteration in range(1000):
    r = y - X.dot(w)
    for j in range(len(w)):
        r = r + X[:, j] * w[j]
        w[j] = X[:, j].dot(r)
        r = r - X[:, j] * w[j]
 
print(w)

Сравните результаты с реализацией линейной регрессии из библиотеки sklearn:

In [ ]:
from sklearn.linear_model import LinearRegression
 
model = LinearRegression(fit_intercept=False)
model.fit(X, y)
 
print(model.coef_)

Если вы все сделали верно, они должны практически совпасть!

## Стохастический градиентный спуск (6 баллов)

**Отмасштабируйте столбцы исходной матрицы *X* (которую мы не нормализовали еще!). Для того, чтобы это сделать, надо вычесть из каждого значения среднее и разделить на стандартное отклонение** (0.5 баллов)

In [34]:
#ваш код 

X1 = (x - np.mean(x)) / np.std(x)


**Добавим единичный столбец**

In [35]:
X1 = np.hstack([np.ones(X1.shape[0]).reshape(-1, 1), X1])

**Создайте функцию mse_error для вычисления среднеквадратичной ошибки, принимающую два аргумента: реальные значения и предсказывающие, и возвращающую значение mse** (0.5 балла)

In [57]:
#ваш код
def mse_error(actual, predicted):
    differences = np.subtract(actual, predicted)
    squared_differences = np.square(differences)
    return squared_differences.mean()


**Сделайте наивный прогноз: предскажите продажи средним значением. После этого рассчитайте среднеквадратичную ошибку для этого прогноза** (0.5 балла)

In [ ]:
#ваш код
train=y[0:180]
test=y[180:]
predicted = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15 ,16 ,17, 18, 19, 20])
predicted[...] = np.mean(y)
mse_error(test, predicted)



**Создайте функцию *lin_pred*, которая может по матрице предикторов *X* и вектору весов линейной модели *w* получить вектор прогнозов** (0.5 балла)

In [ ]:
import random
#функция инициации дополнительных переменных
def initialize(dim): 
  b = np.random.rand()
  theta=np.random.rand(dim)
  return b,theta
b,theta=initialize(5)

print('Bias: ',b,'Weights: ',theta)
#написание функции
def lin_pred(b,theta,X): 
  return b + np.dot(X,theta)




**Создайте функцию *stoch_grad_step* для реализации шага стохастического градиентного спуска. (1.5 балла) 
Функция должна принимать на вход следующие аргументы:**
* матрицу *X*
* вектора *y* и *w*
* число *train_ind* - индекс объекта обучающей выборки (строки матрицы *X*), по которому считается изменение весов
* число *$\eta$* (eta) - шаг градиентного спуска

Результатом будет вектор обновленных весов

Шаг для стохастического градиентного спуска выглядит следующим образом:

$$\Large w_j \leftarrow w_j - \frac{2\eta}{\ell} \sum_{i=1}^\ell{{x_{ij}((w_0 + w_1x_{i1} + w_2x_{i2} +  w_3x_{i3}) - y_i)}}$$

Для того, чтобы написать функцию, нужно сделать следующее:
    
*  посчитать направление изменения: умножить объект обучающей выборки на 2 и на разницу между предсказанным значением и реальным, а потом поделить на количество элементов в выборке.
* вернуть разницу между вектором весов и направлением изменения, умноженным на шаг градиентного спуска

In [ ]:
#ваш код
#функция затрат для дальнейшего анализа
import math
def get_cost(Y,Y_hat):
  Y_resd=Y-Y_hat
  return np.sum(np.dot(Y_resd.T,Y_resd))/len(Y-Y_resd)

Y_hat=lin_pred(b,theta,X1)

get_cost(y,Y_hat)

#Обновим тета для демонстрации изменчивости

def stoch_grad_step(x,y,y_hat,b_0,theta_o,learning_rate):
  db=(np.sum(y_hat-y)*2)/len(y)
  dw=(np.dot((y_hat-y),x)*2)/len(y)
  b_1=b_0-learning_rate*db
  theta_1=theta_o-learning_rate*dw
  return b_1,theta_1

print("After initialization -Bias: ",b,"theta: ",theta)

Y_hat=lin_pred(b,theta,X1)

b,theta=stoch_grad_step(X,y,Y_hat,b,theta,0.01)

print("After first update -Bias: ",b,"theta: ",theta)

get_cost(y,Y_hat)

**Создайте функцию *stochastic_gradient_descent*, для реализации стохастического градиентного спуска (2.5 балла)**

**Функция принимает на вход следующие аргументы:**
- Матрицу признаков X
- Целевую переменнную
- Изначальную точку (веса модели)
- Параметр, определяющий темп обучения
- Максимальное число итераций
- Евклидово расстояние между векторами весов на соседних итерациях градиентного спуска,при котором алгоритм прекращает работу 

**На каждой итерации в вектор (список) должно записываться текущее значение среднеквадратичной ошибки. Функция должна возвращать вектор весов $w$, а также вектор (список) ошибок.**

Алгоритм сследующий:
    
* Инициализируйте расстояние между векторами весов на соседних итерациях большим числом (можно бесконечностью)
* Создайте пустой список для фиксации ошибок
* Создайте счетчик итераций
* Реализуйте оновной цикл обучения пока расстояние между векторами весов больше того, при котором надо прекратить работу (когда расстояния станут слишком маленькими - значит, мы застряли в одном месте) и количество итераций меньше максимально разрешенного: сгенерируйте случайный индекс, запишите текущую ошибку в вектор ошибок, запишите в переменную текущий шаг стохастического спуска с использованием функции, написанной ранее. Далее рассчитайте текущее расстояние между векторами весов и прибавьте к счетчику итераций 1.
* Верните вектор весов и вектор ошибок

In [ ]:
# ваш код
def stochastic_gradient_descent(X, Y, alpha, num_iterations):
  b,theta=initialize(X.shape[1])
  gd_iterations_df=pd.DataFrame(columns=['iteration','cost'])
  for i in range(num_iterations):
      Y_hat = lin_pred(b,theta,X)
      this_cost = get_cost(Y,Y_hat)
      prev_b = b
      prev_theta = theta
      b,theta = stoch_grad_step(X,Y,Y_hat,prev_b,prev_theta,alpha)
      if i % 10==0:
         print('Iteration: ', i, 'Cost: ', this_cost)
         i += 1
         return gd_iterations_df,b,theta
print('Final Estimate of b and theta : ',b,theta) 

gd_iterations_df,b,theta=stochastic_gradient_descent(X1,y,0.001,num_iterations=100)


 **Запустите $10^5$ итераций стохастического градиентного спуска. Укажите вектор начальных весов, состоящий из нулей. Можете поэкспериментировать с параметром, отвечающим за темп обучения.**

**Постройте график зависимости ошибки от номера итерации**

In [ ]:
import matplotlib.pyplot as plt
# ваш код
%matplotlib inline
alpha_df_1,b,theta=stochastic_gradient_descent(X,Y,alpha=0.01,num_iterations=10000)
alpha_df_2,b,theta=stochastic_gradient_descent(X,Y,alpha=0.001,num_iterations=10000)
plt.plot(alpha_df_1['iteration'],alpha_df_1['cost'],label='alpha=0.01')
plt.plot(alpha_df_2['iteration'],alpha_df_2['cost'],label='alpha=0.001')
plt.legend()
plt.ylabel('Ошибка')
plt.xlabel('Номер итерации')
plt.title('Ошибка VS. Номер итерации')


**Выведите вектор весов, к которому сошелся метод.**

In [200]:
# ваш код
gd_iterations_df[99990:100000]
b_new = 0.16248876587930222
theta_new = np.array([0.59323308, 0.10119546, 0.13143591, 0.65597268, 0.28990577])

**Выведите среднеквадратичную ошибку на последней итерации.**

In [ ]:
# ваш код
y_pred = lin_pred(b_new, theta_new, X1)
mse_error(y, y_pred)